In [2]:
import ccxt
import os
from dotenv import load_dotenv
import time
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo

In [3]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')


In [5]:
# Initialize the KuCoin exchange object
kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})
symbol = 'BTCUSD'
symbol_str = "STRKUSDTM"

# Helping Function

In [20]:

import json

from kucoin_futures.client import Trade

def get_pos_size(sy):        
    client = Trade(key=kC_KEY, secret=kC_SECRET, passphrase=kC_PASSWORD)
    positions = client.get_position_details(symbol=sy)
    return positions['currentQty'],positions['isOpen']

def ask_bid():
    order_book = phemex.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]
get_pos_size(sy=symbol_str)


(2, True)

# Daily SMA 20 Days

In [7]:
 #! FIND DAILY SMA PER 20 DAYS

def sma_20d():
    print("starting 20 Days ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
    data_20days.loc[data_20days['SMA_20'] > bid, 'SIGNAL'] = 'SELL'
    data_20days.loc[data_20days['SMA_20'] < bid, 'SIGNAL'] = 'BUY'

    if data_20days['SIGNAL'].iloc[-1] == 'SELL':
        print("Opening as sell ...")
    else:
        print('Opening as Buy ...')
    
    return data_20days

sma_20d()


starting 20 Days ...
Opening as Buy ...


,open,high,low,close,volume,date,SMA_20,SIGNAL
0,64111.9,64484.4,63919.6,64224.3,12399096.0,2024-06-22,NaN,NaN
1,64220.5,64475.0,63082.3,63133.5,34814744.0,2024-06-23,NaN,NaN
2,63139.7,63290.0,58206.9,60223.7,232487908.0,2024-06-24,NaN,NaN
3,60220.8,62372.2,60193.2,61761.2,80158692.0,2024-06-25,NaN,NaN
4,61761.2,62449.0,60660.9,60790.0,58474523.0,2024-06-26,NaN,NaN
...,...,...,...,...,...,...,...,...
95,64316.2,64917.0,62900.0,63119.1,106151622.0,2024-09-25,59895.090,BUY
96,63126.8,65843.1,62650.0,65175.6,126242249.0,2024-09-26,60451.990,BUY
97,65175.6,66490.8,64828.6,65742.1,97440115.0,2024-09-27,61027.830,BUY
98,65751.0,66261.0,65387.2,65843.5,24417441.0,2024-09-28,61574.185,BUY


#  15 Minutes SMA

In [8]:
 #! Define the SMA for the 15 Minutes With Prices bp_1, bp_2, sp_1, sp_2

def sma_15min():
    print("starting sma 15 Minuts  ...")

    timeframe = '15m'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_15m'] = data_15m['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
        
    data_15m['bp_1'] = data_15m['SMA_15m'] * 1.001
    data_15m['bp_2'] = data_15m['SMA_15m'] * 0.997
    data_15m['sp_1'] = data_15m['SMA_15m'] * 0.999
    data_15m['sp_2'] = data_15m['SMA_15m'] * 1.003

    return data_15m

sma_15min()

starting sma 15 Minuts  ...


,open,high,low,close,volume,date,SMA_15m,bp_1,bp_2,sp_1,sp_2
0,65580.1,65624.4,65555.0,65555.3,296320.0,2024-09-29 06:00:00,NaN,NaN,NaN,NaN,NaN
1,65555.3,65576.8,65500.1,65576.8,98241.0,2024-09-29 06:15:00,NaN,NaN,NaN,NaN,NaN
2,65581.8,65581.8,65500.1,65521.7,42253.0,2024-09-29 06:30:00,NaN,NaN,NaN,NaN,NaN
3,65521.8,65527.3,65470.4,65482.4,84298.0,2024-09-29 06:45:00,NaN,NaN,NaN,NaN,NaN
4,65481.9,65546.8,65478.4,65531.5,264371.0,2024-09-29 07:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,64446.0,64519.7,64445.6,64518.9,178505.0,2024-09-30 05:45:00,64594.760,64659.354760,64400.975720,64530.165240,64788.544280
96,64523.0,64714.2,64523.0,64633.4,1203908.0,2024-09-30 06:00:00,64562.425,64626.987425,64368.737725,64497.862575,64756.112275
97,64633.5,64647.9,64510.0,64522.4,927661.0,2024-09-30 06:15:00,64524.960,64589.484960,64331.385120,64460.435040,64718.534880
98,64528.2,64553.1,64442.0,64442.0,1264454.0,2024-09-30 06:30:00,64489.255,64553.744255,64295.787235,64424.765745,64682.722765


# Bot

In [13]:
def bot():
    data_15m = sma_15min()
    data_20d = sma_20d()
    ask,bid = ask_bid()

    # * Define OPEN Order
    signal = data_20d.iloc[-1]['SIGNAL']
    
    open_size = 1
    
    in_pos = pnl_close()[1]


    if not in_pos:
        if signal == "BUY":
            print("Make order as BUY !!")
            bp_1 = data_15m.iloc[-1]['bp_1']
            bp_2 = data_15m.iloc[-1]['bp_2']
            print(f"bp_1: {bp_1} bp_2: {bp_2}")
            # TODO: Make Buy Order Here ...
        else:
            print("Make order as SELL !!")
            sp_1 = data_15m.iloc[-1]['sp_1']
            sp_2 = data_15m.iloc[-1]['sp_2']
            print(f"sp_1: {sp_1} sp_2: {sp_2}")
            # TODO: Make Sell Order Here ... 
    else:
        print("Already In Position, not making new Position")
bot()

starting sma 15 Minuts  ...
starting 20 Days ...
Opening as Buy ...


NameError: name 'pnl_close' is not defined

In [21]:
target = 25

def kill_switch():
    print("Starting the Kill switching ...")
    # TODO: Implement the Kill Switch
    pnl_c, _ ,side = pnl_close_kucoin() #! pnl_close, open_pos, Side-long,short
    size,open_p = get_pos_size(sy=symbol_str)

    print(f"open position: {open_p} side: {side} Size postion: {size}")
        

def pnl_close_phemex():
    print("🔍 Checking to see if it's time to exit on Phemex...")
    
    # Example of different parameters
    params = {'type': 'future', 'currency': 'BTC'}
    poc_dic = phemex.fetch_positions(params=params)[0]
    side = poc_dic['side']
    entry_price = float(poc_dic['info']['avgEntryPrice'])
    leverage = float(poc_dic['leverage'])
    current_price = ask_bid()[1]

    if side == 'long':
        diff = current_price - entry_price
    else:
        diff = entry_price - current_price
    
    try:
        perc = ((diff / entry_price) * leverage)
    except ZeroDivisionError:
        perc = 0

    return f"📊 Difference is {diff} and Percentage: {perc:.2f}%"

def pnl_close_kucoin():
    print("Starting PNL ...")
    
    # Example of different parameters

    poc_dic = kucoin.fetch_ticker(symbol='STRK/USDT')
    perc = poc_dic['percentage']
    diff = float(poc_dic['info']['changePrice'])
    current_price = poc_dic['last']
    entry_price = float(poc_dic['open']) 
    side = 'long' if (diff >= 0 and current_price >= entry_price) else 'short'
    
    print(f"Difference: {diff}, Current Price: {current_price}, Entry Price: {entry_price}, Side: {side}, Percentage: {perc}%")


    pnlclose = False
    in_pos = False
    if perc > 0:
        # * We are in a winning position (profit)
        if perc > target:
            # ! Closing the position as target is hit
            print("In Profit also Hit the Target ...")
            pnlclose = True
            kill_switch()
        else:
            print("In Profit but  not hit the target, so wait !!")
    else:
        in_pos = True
        print("We are not in Profit yet.")
    print("Finishing the PNL close ...")

    return pnlclose, in_pos,side

# print(pnl_close_phemex())
# pnl_close_kucoin()
kill_switch()


Starting the Kill switching ...
Starting PNL ...


RequestTimeout: kucoin GET https://api.kucoin.com/api/v1/market/stats?symbol=STRK-USDT

2

[[15, 0, 0, 0, 0],
 [7, 14, 0, 0, 0],
 [6, 8, 13, 0, 0],
 [0, 5, 9, 12, 0],
 [0, 0, 4, 10, 11]]